In [102]:
import matplotlib.pyplot as plt
import pandas as pd

In [103]:
finances = pd.read_pickle('IPL_finances.pkl')
results  = pd.read_pickle('IPL_results.pkl')

#Assume that the current 60crore rupees budget applied forever (rules for 2013 are not clear to me)
finances['Spend']=600000000.0-finances['Balance']
finances['Spend_crores']=finances['Spend']/10000000.0


In [104]:
#Get all teams ever for which we have both results and finance data
teams_results = set(results.index.values)
teams_finances= set(finances.index.get_level_values(level=1))
#Final set of teams with both data
teams = teams_results.intersection(teams_finances)
print(teams)

{'Gujarat Lions', 'Kolkata Knight Riders', 'Royal Challengers Bangalore', 'Mumbai Indians', 'Pune Warriors India', 'Rajasthan Royals', 'Delhi Capitals', 'Chennai Super Kings', 'Rising Pune Supergiant', 'Kings XI Punjab', 'Sunrisers Hyderabad'}


In [105]:
#Currently Active Teams based on results for latest year
currently_active = results.iloc[:,-1].notnull()
currently_active_teams = results[currently_active].index.to_list()
print(currently_active_teams)

['Chennai Super Kings', 'Delhi Capitals', 'Kings XI Punjab', 'Kolkata Knight Riders', 'Mumbai Indians', 'Rajasthan Royals', 'Royal Challengers Bangalore', 'Sunrisers Hyderabad']


In [106]:
#Total money spent ever in IPL (in crores)
sum(finances.Spend_crores)

3454.1495342

In [107]:
#Money spent by teams since 2013
finances.Spend_crores.groupby(level=[0]).sum()

year
2013    389.964534
2014    468.100000
2015    432.645000
2016    417.720000
2017    422.820000
2018    468.700000
2019    441.550000
2020    412.650000
Name: Spend_crores, dtype: float64

In [108]:
#Money spent by teams since 2013
finances.Spend_crores.groupby(level=[1]).sum()

Team
Chennai Super Kings            337.988869
Delhi Capitals                 441.148534
Gujarat Lions                   99.150000
Kings XI Punjab                383.005761
Kolkata Knight Riders          428.396954
Mumbai Indians                 462.150392
Pune Warriors India             55.094807
Rajasthan Royals               277.115046
Rising Pune Supergiant         114.500000
Royal Challengers Bangalore    456.709716
Sunrisers Hyderabad            398.889454
Name: Spend_crores, dtype: float64

In [109]:
def get_games_point(results,team = 'Delhi Capitals',year=2019):
    try:
        res  = results[year].loc[team][0]
        wins   = res[0]
        losses = res[1]
        ties   = res[2]
        nr     = res[3]
        games  = wins+losses+ties+nr
        point = 2*wins+1*(ties+nr)
    except TypeError:
        point = 0.0
        games  = 0
    return(games, point)


In [110]:
points = {}
for year in range(2013,2020):
    for team in teams:
        games,point = get_games_point(results,team=team,year=year)
        points[(year,team)] = [int(games),int(point)]

In [111]:
points = pd.DataFrame.from_dict(points)
points = points.transpose()
points.columns = ['games','points']
points['ppg']=points.points/points.games
points = points.fillna(0)
points.index.set_names(['year', 'Team'], inplace=True)

In [125]:
data1 = points.join(finances)
data1['Spend_crores']= data1['Spend_crores'].fillna(0)
data = data1.drop(columns=['Balance','Overseas','TotalPlayers','Spend'])

In [126]:
data

games  points       ppg  Spend_crores
year Team                                                              
2013 Gujarat Lions                    0       0  0.000000      0.000000
     Kolkata Knight Riders           16      12  0.750000     50.796954
     Royal Challengers Bangalore     16      18  1.125000     55.659716
     Mumbai Indians                  16      22  1.375000     57.915392
     Pune Warriors India             16       8  0.500000     55.094807
     Rajasthan Royals                16      20  1.250000     16.865046
     Delhi Capitals                  16       6  0.375000     55.998534
     Chennai Super Kings             16      22  1.375000     48.738869
     Rising Pune Supergiant           0       0  0.000000      0.000000
     Kings XI Punjab                 16      16  1.000000     22.955761
     Sunrisers Hyderabad             16      20  1.250000     25.939454
2014 Gujarat Lions                    0       0  0.000000      0.000000
     Kolkata Knight Riders           14      18  1.285714     59.000000
     Royal Challengers Bangalore     14      10  0.714286     60.000000
     Mumbai Indians                  14      14  1.000000     59.950000
     Pune Warriors India              0       0  0.000000      0.000000
     Rajasthan Royals                14      14  1.000000     54.450000
     Delhi Capitals                  14       4  0.285714     60.000000
     Chennai Super Kings             14      18  1.285714     59.800000
     Rising Pune Supergiant           0       0  0.000000      0.000000
     Kings XI Punjab                 14      22  1.571429     55.900000
     Sunrisers Hyderabad             14      12  0.857143     59.000000
2015 Gujarat Lions                    0       0  0.000000      0.000000
     Kolkata Knight Riders           14      15  1.071429     50.800000
     Royal Challengers Bangalore     14      16  1.142857     58.000000
     Mumbai Indians                  14      16  1.142857     55.095000
     Pune Warriors India              0       0  0.000000      0.000000
     Rajasthan Royals                14      16  1.142857     49.350000
     Delhi Capitals                  14      11  0.785714     58.650000
     Chennai Super Kings             14      18  1.285714     59.300000
...                                 ...     ...       ...           ...
2017 Mumbai Indians                  14      20  1.428571     56.645000
     Pune Warriors India              0       0  0.000000      0.000000
     Rajasthan Royals                 0       0  0.000000      0.000000
     Delhi Capitals                  14      12  0.857143     50.950000
     Chennai Super Kings              0       0  0.000000      0.000000
     Rising Pune Supergiant          14      18  1.285714     59.700000
     Kings XI Punjab                 14      14  1.000000     46.100000
     Sunrisers Hyderabad             14      17  1.214286     47.750000
2018 Gujarat Lions                    0       0  0.000000      0.000000
     Kolkata Knight Riders           14      16  1.142857     60.000000
     Royal Challengers Bangalore     14      12  0.857143     59.850000
     Mumbai Indians                  14      12  0.857143     59.350000
     Pune Warriors India              0       0  0.000000      0.000000
     Rajasthan Royals                14      14  1.000000     58.350000
     Delhi Capitals                  14      10  0.714286     58.400000
     Chennai Super Kings             14      18  1.285714     53.500000
     Rising Pune Supergiant           0       0  0.000000      0.000000
     Kings XI Punjab                 14      12  0.857143     59.900000
     Sunrisers Hyderabad             14      18  1.285714     59.350000
2019 Gujarat Lions                    0       0  0.000000      0.000000
     Kolkata Knight Riders           14      12  0.857143     53.950000
     Royal Challengers Bangalore     14      11  0.785714     58.200000
     Mumbai Indians                  14      18  1.285714     56.450000
  